# Rozdział 3 — Skończone MDP (Robot + Gridworld + opcjonalnie FrozenLake) (wersja pełna)

Lektura: Sutton & Barto, rozdz. 3.

**Cel rozdziału:** nauczyć się *modelować* środowisko jako MDP i liczyć wartości **dokładnie** (gdy znamy model przejść).

W tym notebooku rozróżniamy dwa tryby:
- **Ewaluacja polityki**: liczymy $v_\pi$ dla *zadanej* polityki $\pi$.
- **Optymalizacja**: szukamy $v_*$ i $\pi_*$ (w tym rozdziale głównie konceptualnie / na małych przykładach).


> **Notatki dla prowadzącego:**
> - W tym rozdziale kluczowe jest rozróżnienie: **model świata** (parametry, przejścia) vs **polityka** (wybór akcji).
> - Najpierw robimy *ewaluację* (bez max), a dopiero potem pokazujemy, gdzie wchodzi *optymalność* (max/argmax).


In [1]:
# --- Setup (importy + helpery) ---
import numpy as np

def evaluate_policy_linear_system(P_pi: np.ndarray, r_pi: np.ndarray, gamma: float) -> np.ndarray:
    """Rozwiązuje równanie Bellmana dla danej polityki w postaci macierzowej.

    Dla polityki pi mamy:
        v = r_pi + gamma * P_pi * v
    czyli:
        (I - gamma * P_pi) v = r_pi

    Zwraca wektor v (shape: [nS]).
    """
    nS = P_pi.shape[0]
    I = np.eye(nS)
    return np.linalg.solve(I - gamma * P_pi, r_pi)

def pretty_matrix_as_grid(v: np.ndarray, nrow: int, ncol: int, decimals: int = 1):
    """Pomocniczo: wyświetl wektor wartości jako siatkę (nrow x ncol)."""
    grid = v.reshape(nrow, ncol)
    with np.printoptions(precision=decimals, suppress=True):
        print(grid)

def action_arrows(pi_det: np.ndarray, nrow: int, ncol: int):
    """Zamienia deterministyczną politykę (akcja w każdym stanie) na strzałki w siatce."""
    arrows = {0:'↑', 1:'→', 2:'↓', 3:'←', None:'·'}
    out = []
    for r in range(nrow):
        row = []
        for c in range(ncol):
            s = r*ncol + c
            a = int(pi_det[s]) if pi_det[s] is not None else None
            row.append(arrows.get(a, '?'))
        out.append(' '.join(row))
    print('\n'.join(out))


## 0. Wspólny interfejs MDP

Używamy konwencji zgodnej z `gymnasium`:

- `P[s][a]` to lista wyników `(p, s2, r, terminated)`
- `pi[s,a]` to prawdopodobieństwo wyboru akcji `a` w stanie `s`

Z tego budujemy:
- macierz przejść polityki $P_\pi$ (rozmiar $nS\times nS$)
- wektor nagród $r_\pi$ (rozmiar $nS$)

Takie $P_\pi$ i $r_\pi$ pozwalają rozwiązać $v_\pi$ przez algebrę liniową.

In [2]:
# Budowa P_pi oraz r_pi (rozwiązanie)
def build_P_r_for_policy(P, pi):
    """Buduje (P_pi, r_pi) dla zadanej polityki pi.

    P: dict, P[s][a] -> list (p, s2, r, terminated)
    pi: ndarray (nS, nA), pi[s,a] = P(A_t=a | S_t=s)

    Uwaga o terminated:
    - Jeśli przejście jest terminalne, to w równaniu wartości przyszła wartość jest 0,
      więc NIE dodajemy wkładu do P_pi (bo i tak mnożyłby V(s2)=0).
    """
    nS = len(P)
    # nA bierzemy z pierwszego stanu
    nA = len(P[0])
    P_pi = np.zeros((nS, nS), dtype=float)
    r_pi = np.zeros(nS, dtype=float)

    for s in range(nS):
        for a in range(nA):
            w = float(pi[s, a])
            if w == 0.0:
                continue
            outcomes = P[s][a]
            if not outcomes:
                continue
            for (p, s2, r, terminated) in outcomes:
                r_pi[s] += w * p * float(r)
                if not terminated:
                    P_pi[s, int(s2)] += w * p

    return P_pi, r_pi


In [ ]:
# Budowa P_pi oraz r_pi (rozwiązanie)
def build_P_r_for_policy(P, pi):
    """
    Buduje (P_pi, r_pi) dla zadanej polityki π.

    Wejście:
    - P: model środowiska w formacie
         P[s][a] -> lista (p, s2, r, terminated)
    - pi: polityka stochastyczna w postaci macierzy (nS, nA),
          pi[s, a] = P(A_t = a | S_t = s)

    Wyjście:
    - P_pi: macierz przejść dla polityki π (nS x nS)
    - r_pi: wektor nagród oczekiwanych dla polityki π (nS,)

    Sens:
    Z ogólnego MDP (P[s][a]) robimy „świat widziany przez politykę π”,
    potrzebny do rozwiązania równania:
        v = r_pi + γ P_pi v
    """
    nS = len(P)              # liczba stanów
    nA = len(P[0])           # liczba akcji (z pierwszego stanu)

    # Macierz przejść i wektor nagród dla polityki π
    P_pi = np.zeros((nS, nS), dtype=float)
    r_pi = np.zeros(nS, dtype=float)

    # Iterujemy po wszystkich stanach
    for s in range(nS):

        # Iterujemy po wszystkich akcjach
        for a in range(nA):

            # Waga akcji a w stanie s wg polityki π
            w = float(pi[s, a])

            # Jeśli polityka nigdy nie wybiera tej akcji, pomijamy ją
            if w == 0.0:
                continue

            outcomes = P[s][a]

            # Jeśli akcja jest niedostępna (pusta lista), pomijamy
            if not outcomes:
                continue

            # Iterujemy po wszystkich możliwych skutkach akcji a w stanie s
            for (p, s2, r, terminated) in outcomes:

                # Składnik nagrody oczekiwanej:
                # r_pi[s] = E_π[R_{t+1} | S_t = s]
                r_pi[s] += w * p * float(r)

                # Składnik przejść:
                # P_pi[s, s2] = P(S_{t+1} = s2 | S_t = s, π)
                #
                # Jeśli przejście jest terminalne:
                # - V(s2) = 0 w równaniu Bellmana,
                # - więc NIE dodajemy go do macierzy P_pi
                if not terminated:
                    P_pi[s, int(s2)] += w * p

    return P_pi, r_pi


> **Notatki dla prowadzącego (Ćw. 0.1):**
> - To ćwiczenie jest kluczowe: pokazuje, jak przejść od opisu świata $P$ i zachowania $\pi$ do macierzy $P_\pi$.
> - Warto podkreślić różnicę: **ewaluacja polityki** = uśrednianie po akcjach (bez max).
> - Typowa pułapka: akcje niedostępne (puste listy) oraz obsługa `terminated`.


## A. Recycling Robot (mały MDP z książki)

To jest 2-stanowy przykład, idealny do:
- ćwiczenia budowy modelu `P`,
- policzenia $v_\pi$ dokładnie,
- pokazania, że zmiana parametrów świata może zmienić $\pi_*$.

**Parametry (ściąga):** `alpha` (bezpieczeństwo SEARCH w H), `beta` (bezpieczeństwo SEARCH w L), `r_search`, `r_wait`, `rescue_cost` (kara za rozładowanie w L), `gamma`.


### Ćwiczenie A1.1 — Zbuduj model `P` dla robota recyklingowego

Zaimplementuj `build_recycling_robot_P(...)`.

Konwencje:
- stany: `0=H`, `1=L`
- akcje: `0=SEARCH`, `1=WAIT`, `2=RECHARGE`
- jeśli akcja jest niedostępna w danym stanie (np. RECHARGE w H), zwróć pustą listę `[]`.


To jest klasyczny przykład „recycling robot” z Sutton & Barto (rozdz. 3).

**Stany**
- `H` (wysoka energia)
- `L` (niska energia)

**Akcje**
- `search`: zbiera puszki (dobra nagroda, ale może rozładować baterię)
- `wait`: czeka (mała nagroda)
- `recharge`: tylko w stanie `L` (wraca do `H` z nagrodą 0)

Parametry jak w książce:
- Z `H`, gdy wybierzemy `search`: zostaje w `H` z prawd. $\alpha$, przechodzi do `L` z prawd. $1-\alpha$
- Z `L`, gdy wybierzemy `search`: zostaje w `L` z prawd. $\beta$ i dostaje $r_{search}$; z prawd. $1-\beta$ robot się „rozładowuje”, jest odnaleziony i naładowany do `H` z nagrodą $-3$
- `wait` utrzymuje poziom energii (nagroda $r_{wait}$)
- `recharge` z `L` przechodzi do `H` z nagrodą $0$

Model zapisujemy w stylu Gymnasium:
`P[s][a] -> lista (prob, s2, r, terminated)`.



**Podsumowanie przejść**

Niech `H=0`, `L=1` oraz `search=0`, `wait=1`, `recharge=2`.

| stan $s$ | akcja $a$ | następny stan $s'$ | prawdopodobieństwo $p$ | nagroda $r$ |
|---|---|---|---|---|
| H | search | H | $\alpha$ | $r_{search}$ |
| H | search | L | $1-\alpha$ | $r_{search}$ |
| H | wait | H | $1$ | $r_{wait}$ |
| L | search | L | $\beta$ | $r_{search}$ |
| L | search | H | $1-\beta$ | `rescue_cost` (w książce: $-3$) |
| L | wait | L | $1$ | $r_{wait}$ |
| L | recharge | H | $1$ | $0$ |

Traktujemy `recharge` w stanie `H` jako akcję **niedostępną**.

In [10]:
def build_recycling_robot_P(alpha=0.8, beta=0.4, r_search=5.0, r_wait=1.0, rescue_cost=-3.0):
    """Return (P, nS, nA) for Recycling Robot.

    States: 0=H, 1=L
    Actions: 0=SEARCH, 1=WAIT, 2=RECHARGE (only in L)

    P[s][a] -> list (p, s2, r, terminated)
    """
    # 1) Rozmiar MDP: 2 stany (H,L) i 3 akcje (SEARCH, WAIT, RECHARGE)
    nS, nA = 2, 3
    # 2) Inicjalizacja pustej struktury przejść
    #    P[s][a] = [] oznacza brak zdefiniowanych przejść (np. akcja niedostępna)
    P = {s: {a: [] for a in range(nA)} for s in range(nS)}
    # 3) Aliasowanie indeksów dla czytelności

    H, L = 0, 1
    SEARCH, WAIT, RECHARGE = 0, 1, 2
    # -----------------------------
    # Stan H (wysoka energia)
    # -----------------------------
    # SEARCH w H: nagroda r_search; przejście do H z p=alpha, do L z p=1-alpha
    
    # High energy (H)
    P[H][SEARCH] = [
        (alpha, H, r_search, False),
        (1 - alpha, L, r_search, False),
    ]
    # WAIT w H: zawsze zostajemy w H; nagroda r_wait
    P[H][WAIT] = [
        (1.0, H, r_wait, False),
    ]
    # RECHARGE w H: akcja niedostępna w tym stanie
    P[H][RECHARGE] = []  # not available in H

    # -----------------------------
    # Stan L (niska energia)
    # -----------------------------
    # SEARCH w L:
    #   - z p=beta: zostajemy w L i dostajemy r_search
    #   - z p=1-beta: rozładowanie -> powrót do H i kara rescue_cost
    # Low energy (L)
    P[L][SEARCH] = [
        (beta, L, r_search, False),
        (1 - beta, H, rescue_cost, False),
    ]
    # WAIT w L: zawsze zostajemy w L; nagroda r_wait
    P[L][WAIT] = [
        (1.0, L, r_wait, False),
    ]
    # RECHARGE w L: zawsze przechodzimy do H; nagroda 0
    P[L][RECHARGE] = [
        (1.0, H, 0.0, False),
    ]

    return P, nS, nA

# --------------------------------------------------
# Testy poprawności modelu
# --------------------------------------------------

# --- testy ---
P, nS, nA = build_recycling_robot_P(alpha=0.8, beta=0.4, r_search=5.0, r_wait=1.0, rescue_cost=-3.0)

assert nS == 2 and nA == 3
assert abs(sum(p for p, *_ in P[0][0]) - 1.0) < 1e-12  # H, SEARCH
assert abs(sum(p for p, *_ in P[0][1]) - 1.0) < 1e-12  # H, WAIT
assert P[0][2] == []                                  # H, RECHARGE not allowed
assert abs(sum(p for p, *_ in P[1][0]) - 1.0) < 1e-12  # L, SEARCH
assert abs(sum(p for p, *_ in P[1][1]) - 1.0) < 1e-12  # L, WAIT
assert abs(sum(p for p, *_ in P[1][2]) - 1.0) < 1e-12  # L, RECHARGE

print("A1.1 tests passed ✅")


A1.1 tests passed ✅


In [4]:
P

{0: {0: [(0.8, 0, 5.0, False), (0.19999999999999996, 1, 5.0, False)],
  1: [(1.0, 0, 1.0, False)],
  2: []},
 1: {0: [(0.4, 1, 5.0, False), (0.6, 0, -3.0, False)],
  1: [(1.0, 1, 1.0, False)],
  2: [(1.0, 0, 0.0, False)]}}

In [5]:
nS

2

In [6]:
nA

3

In [19]:
(P[0][1])

[(1.0, 0, 1.0, False)]

In [15]:
(sum(p for p, *_ in P[0][1]))

1.0

> **Notatki dla prowadzącego (A1.1):**
> - Te testy sprawdzają głównie **poprawność probabilistyczną** (sumy prawdopodobieństw = 1) i akcje niedostępne.
> - Warto dopowiedzieć: to jest dokładnie obiekt $p(s',r\mid s,a)$ w równaniach Bellmana.


### Co sprawdzają te testy? (krótkie podsumowanie)

Każdy test sprawdza, czy dla **każdej dostępnej akcji w danym stanie** lista `P[s][a]` opisuje **pełny i poprawny rozkład prawdopodobieństwa**.

Konkretnie:
- `P[s][a]` to lista możliwych wyników po wykonaniu akcji `a` w stanie `s`,
- pierwszy element każdej krotki to **prawdopodobieństwo** tego wyniku,
- testy sumują **tylko te prawdopodobieństwa**,
- ich suma musi wynosić **1**, bo *jeden z tych wyników musi się wydarzyć*.

Akcje niedostępne (np. `RECHARGE` w stanie `H`) są reprezentowane przez pustą listę i nie podlegają temu sprawdzeniu.

---

### Jednozdaniowe podsumowanie (na zajęcia)

Testy sprawdzają, czy każda dostępna akcja w danym stanie definiuje pełny i poprawny rozkład prawdopodobieństwa, bo tylko wtedy równania Bellmana mają sens.

---

### Co można dodać dalej?

- test semantyczny: czy **przejścia i nagrody** odpowiadają opisowi zadania  
  (np. czy `RECHARGE` w `L` zawsze prowadzi do `H` z nagrodą `0`),
- testy porównujące wartości `Q(s,a)` dla różnych akcji w jednym stanie.


### Ćwiczenie A1.2 — Ewaluacja zadanej polityki $v_\pi$

Zdefiniuj politykę i policz $v_\pi$ przez algebrę liniową.

Wskazówka: polityka stochastyczna to macierz `pi[s,a]` (wiersze sumują się do 1 po dostępnych akcjach).


In [63]:
gamma = 0.9
P, nS, nA = build_recycling_robot_P()

H, L = 0, 1
SEARCH, WAIT, RECHARGE = 0, 1, 2

# Polityka 1: w H -> SEARCH, w L -> RECHARGE
pi1 = np.zeros((nS, nA))
pi1[H, SEARCH] = 1.0
pi1[L, RECHARGE] = 1.0

# Polityka 2: w H -> WAIT, w L -> WAIT
pi2 = np.zeros((nS, nA))
pi2[H, RECHARGE] = 1.0
pi2[L, SEARCH] = 1.0

P_pi1, r_pi1 = build_P_r_for_policy(P, pi1)
v1 = evaluate_policy_linear_system(P_pi1, r_pi1, gamma)

P_pi2, r_pi2 = build_P_r_for_policy(P, pi2)
v2 = evaluate_policy_linear_system(P_pi2, r_pi2, gamma)

print("v_pi1(H), v_pi1(L) =", np.round(v1, 4))
print("v_pi2(H), v_pi2(L) =", np.round(v2, 4))


v_pi1(H), v_pi1(L) = [42.3729 38.1356]
v_pi2(H), v_pi2(L) = [0.     0.3125]


> **Notatki dla prowadzącego (A1.2):**
> - To jest *policy evaluation*: bez żadnego `max`. Liczymy dokładnie $v_\pi$.
> - Zwróć uwagę, że `pi` to część agenta, a `P` to część świata.


## Jak rozumieć ten przykład? (policy evaluation)

### Co jest czym?
- **Model `P`**: opis świata — co może się stać po wykonaniu danej akcji w danym stanie  
  (przejścia i nagrody).
- **Polityka `π`**: reguła wyboru akcji w każdym stanie  
  (czyli: *co robot robi w `H` i w `L`*).
- **Funkcja wartości `v_π`**: mówi, ile średnio zyskujemy w przyszłości,
  jeśli **zawsze** stosujemy daną politykę.

---

### Co robi ten kod?
1. Definiujemy **ten sam świat** (`P`).
2. Definiujemy **dwie różne polityki** (`π₁`, `π₂`), czyli dwa proste „instrukcje zachowania” robota.
3. Dla każdej polityki **dokładnie liczymy** jej funkcję wartości `v_π`
   (rozwiązując równania Bellmana dla danej polityki).
4. Porównujemy, **która polityka jest lepsza** i w jakich stanach.

---

### Czego tu NIE robimy?
- Nie szukamy polityki optymalnej.
- Nie używamy maksymalizacji (`max`).
- Nie uczymy się polityki.

To jest **policy evaluation**, a nie **policy optimization**.

---

### Jednozdaniowe podsumowanie (na zajęcia)
> W tym przykładzie porównujemy dwie z góry ustalone polityki w tym samym świecie i liczymy,
> jak dobre są te strategie, zanim zaczniemy je optymalizować.


## A1.3 — Co tu właściwie robimy? (intuicja polityki optymalnej)

Ponieważ robot ma tylko 2 stany, możemy znaleźć najlepszą politykę przez sprawdzenie wszystkich deterministycznych polityk.
To daje intuicję, czym jest $\pi_*$, **bez wprowadzania jeszcze algorytmów DP**.

### Co oznacza „polityka optymalna” w tym przykładzie?
Polityka optymalna $\\pi_*$ to taka polityka, która **daje największą wartość oczekiwaną** (tu: $v(H)$),
spośród wszystkich możliwych polityk.

Formalnie:
$$
\\pi_* = \\arg\\max_{\\pi} v_{\\pi}.
$$

---

### Dlaczego możemy to zrobić „ręcznie”?
Robot ma tylko:
- 2 stany (`H`, `L`),
- skończoną liczbę akcji w każdym stanie.

Liczba **wszystkich deterministycznych polityk** jest bardzo mała (6),
więc możemy:
1. wypisać wszystkie polityki,
2. dla każdej dokładnie policzyć $v_{\\pi}$ (z równań Bellmana),
3. wybrać najlepszą.

To jest **brute force**, ale w tym małym przykładzie jest:
- poprawne,
- przejrzyste,
- bardzo dydaktyczne.

---

### Czego tu jeszcze NIE robimy?
- Nie rozwiązujemy równań optymalności Bellmana z `max` po akcjach.
- Nie używamy algorytmów Dynamic Programming (value iteration, policy iteration).

Zamiast tego:
- **najpierw rozumiemy, czym jest $\\pi_*$**,  
- a dopiero później (w kolejnym rozdziale) uczymy się,
  jak znaleźć $\\pi_*$ bez przeglądania wszystkich polityk.

---



In [64]:
def all_deterministic_policies_robot():
    """
    Generuje wszystkie deterministyczne polityki dla robota recyklingowego.

    Ponieważ:
    - w stanie H dostępne są 2 sensowne akcje (SEARCH, WAIT),
    - w stanie L dostępne są 3 akcje (SEARCH, WAIT, RECHARGE),

    liczba wszystkich deterministycznych polityk wynosi 2 * 3 = 6.

    Każda polityka pi jest macierzą:
        pi[s, a] = 1  -> akcja a jest zawsze wybierana w stanie s
        pi[s, a] = 0  -> akcja a nigdy nie jest wybierana w stanie s
    """
    # Indeksy stanów
    H, L = 0, 1

    # Indeksy akcji
    SEARCH, WAIT, RECHARGE = 0, 1, 2

    policies = []

    # Wybieramy wszystkie możliwe akcje w stanie H
    for aH in [SEARCH, WAIT]:

        # Wybieramy wszystkie możliwe akcje w stanie L
        for aL in [SEARCH, WAIT, RECHARGE]:

            # Tworzymy pustą politykę (2 stany x 3 akcje)
            pi = np.zeros((2, 3))

            # Polityka deterministyczna:
            # w stanie H zawsze wybieramy akcję aH
            pi[H, aH] = 1.0

            # w stanie L zawsze wybieramy akcję aL
            pi[L, aL] = 1.0

            # Dodajemy gotową politykę do listy
            policies.append(pi)

    return policies


def best_policy_robot(P, gamma=0.9):
    """
    Wybiera najlepszą politykę spośród wszystkich deterministycznych polityk.

    Kryterium:
    - maksymalizujemy wartość v_pi(H),
      czyli oczekiwany zdyskontowany zwrot,
      jeśli startujemy w stanie H i stosujemy politykę pi.

    Uwaga dydaktyczna:
    - to jest brute force (sprawdzamy wszystkie polityki),
    - działa tylko dlatego, że MDP jest bardzo małe.
    """
    best_pi = None    # najlepsza znaleziona polityka
    best_vH = None    # jej wartość v_pi(H)

    # Iterujemy po wszystkich możliwych politykach
    for pi in all_deterministic_policies_robot():

        # Budujemy model świata "widoczny" dla danej polityki:
        # P_pi   — macierz przejść dla tej polityki
        # r_pi   — wektor nagród dla tej polityki
        P_pi, r_pi = build_P_r_for_policy(P, pi)

        # Liczymy dokładnie funkcję wartości v_pi,
        # rozwiązując układ równań Bellmana:
        # v = r_pi + gamma * P_pi * v
        v = evaluate_policy_linear_system(P_pi, r_pi, gamma)

        # Interesuje nas wartość w stanie H
        vH = float(v[0])

        # Sprawdzamy, czy ta polityka jest lepsza od dotychczasowej
        if best_vH is None or vH > best_vH:
            best_vH = vH
            best_pi = pi

    return best_pi, best_vH


# --------------------------------------------------
# Uruchomienie: znalezienie polityki optymalnej
# --------------------------------------------------

# Budujemy model świata robota
P, nS, nA = build_recycling_robot_P()

# Szukamy najlepszej polityki (brute force)
pi_star, vH_star = best_policy_robot(P, gamma=0.9)

# Mapowanie indeksów akcji na czytelne nazwy
action_name = {0: "SEARCH", 1: "WAIT", 2: "RECHARGE"}

print("Najlepsza polityka (wg v(H)) ma v(H) =", round(vH_star, 4))
print("pi*(H) =", action_name[int(np.argmax(pi_star[0]))])
print("pi*(L) =", action_name[int(np.argmax(pi_star[1]))])


Najlepsza polityka (wg v(H)) ma v(H) = 42.3729
pi*(H) = SEARCH
pi*(L) = RECHARGE


> **Notatki dla prowadzącego (A1.3):**
> - To jest prosty, ale bardzo czysty most do pojęcia $\pi_*$.
> - Podkreśl: tutaj *nie używamy DP*, tylko "brute force" bo MDP jest malutkie.



### Jednozdaniowe podsumowanie (na zajęcia)
> W tym ćwiczeniu znajdujemy politykę optymalną przez sprawdzenie wszystkich możliwych strategii,
> co daje intuicję, czym jest $\\pi_*$, zanim poznamy algorytmy Dynamic Programming.

Notatka dotycząca - continuing vs episodic tasks:

## Uwaga: zadanie ciągłe (continuing) vs epizodyczne (episodic)

Ten przykład robota recyklingowego jest **zadaniem ciągłym (continuing task)**.

### Co to znaczy w praktyce?
- Nie ma stanu terminalnego.
- Robot działa **bez końca**.
- Funkcja wartości $v_\pi(s)$ opisuje **zdyskontowaną sumę nagród w nieskończonej przyszłości**:
  $$
  v_\pi(s) = \mathbb{E}\left[\sum_{t=0}^{\infty} \gamma^t R_{t+1}\right].
  $$

Dlatego wartości, takie jak `v(H) ≈ 42`, mogą być **znacznie większe niż pojedyncze nagrody** (np. `r_search = 5`).

---

### Dla porównania: zadanie epizodyczne
W zadaniu epizodycznym:
- istnieje stan terminalny,
- epizod się kończy,
- suma nagród jest skończona **nawet bez dyskontowania**,
- często można przyjąć $\gamma = 1$.

Przykład: klasyczny gridworld z polem terminalnym.

---

### Dlaczego tu potrzebujemy $\gamma < 1$?
W zadaniu ciągłym:
- bez dyskontowania ($\gamma = 1$),
- suma nagród mogłaby być nieskończona.

Dyskontowanie:
- zapewnia, że $v_\pi(s)$ jest skończone,
- formalizuje fakt, że **nagrody bliższe w czasie są ważniejsze**.

---

### Jednozdaniowe podsumowanie (na zajęcia)
> Robot recyklingowy to zadanie ciągłe: nie ma końca epizodu, więc funkcja wartości mierzy długoterminowy, zdyskontowany zysk, a nie wynik pojedynczego epizodu.


## A1.4 — Sweep parametrów środowiska: kiedy zmienia się π*(L)?
Teraz zmieniamy `beta` i `rescue_cost` i obserwujemy, jak zmienia się optymalna decyzja w stanie `L`.
To pokazuje, że **parametry świata** wpływają na to, jaka polityka jest najlepsza.

### Cel tego eksperymentu
Celem jest pokazanie, że **polityka optymalna nie jest stała**, lecz zależy od
**parametrów świata (MDP)**.

Nie zmieniamy algorytmu ani definicji optymalności —
zmieniamy tylko to, *jak działa środowisko*,
i obserwujemy, jak zmienia się najlepsza decyzja robota w stanie `L`.

---

### Co jest stałe, a co zmieniamy?
**Stałe:**
- `alpha = 0.8` — SEARCH w `H` jest dość bezpieczne
- `r_search = 5`, `r_wait = 1`
- `gamma = 0.9` (zadanie ciągłe)

**Zmieniamy:**
- `beta` — jak bezpieczne jest SEARCH w stanie `L`
- `rescue_cost` — jak bolesna jest awaria przy SEARCH w `L`

Dla każdej pary `(beta, rescue_cost)`:
- wyznaczamy politykę optymalną `π*`,
- wypisujemy decyzje w stanach `H` i `L`.

---

### Jak czytać tabelę wyników?
Każdy wiersz odpowiada **innemu światu** (innemu MDP),
ale rozwiązujemy **to samo zadanie optymalizacji**.

Kolumny:
- `beta`, `rescue_cost` — parametry świata,
- `pi*(H)` — najlepsza decyzja w stanie wysokiej energii,
- `pi*(L)` — najlepsza decyzja w stanie niskiej energii.

---

### Najważniejsze obserwacje
1. **`π*(H)` jest zawsze `SEARCH`.**  
   Przy tych parametrach opłaca się szukać, gdy energia jest wysoka.

2. **`π*(L)` zmienia się zależnie od parametrów świata.**
   - Małe `beta` (SEARCH w `L` jest ryzykowne) → `RECHARGE`
   - Bardzo ujemny `rescue_cost` (duża kara) → `RECHARGE`
   - Duże `beta` (SEARCH w `L` prawie zawsze się udaje) → `SEARCH`

3. Widać **efekt progu**:
   niewielka zmiana parametrów może zmienić najlepszą decyzję w stanie `L`.

---

### Wniosek pojęciowy
Parametry świata **nie są polityką**,  
ale wpływają na **wartości akcji `Q(s,a)`**.

Polityka optymalna `π*` wynika z porównania tych wartości,
więc gdy parametry zmieniają ich ranking,
**zmienia się także `π*`.**

---

### Jednozdaniowe podsumowanie (na zajęcia)
> Ten eksperyment pokazuje, że polityka optymalna nie jest „na stałe”,
> lecz zależy od tego, jak ryzykowne i kosztowne są decyzje w danym świecie.




In [27]:
# A1.4 — Sweep parametrów: kiedy zmienia się π*(L)

alpha = 0.8        # bezpieczeństwo SEARCH w H
r_search = 5.0     # nagroda za SEARCH
r_wait = 1.0       # nagroda za WAIT
gamma = 0.9        # dyskontowanie (zadanie ciągłe)

beta_list = [0.1, 0.3, 0.5, 0.7, 0.9]        # bezpieczeństwo SEARCH w L
rescue_list = [-1.0, -3.0, -6.0, -10.0]     # kara za awarię w L

action_name = {0: "SEARCH", 1: "WAIT", 2: "RECHARGE"}  # mapowanie akcji

print("Ustawienia stałe:", "alpha=", alpha, "r_search=", r_search,
      "r_wait=", r_wait, "gamma=", gamma)
print("beta   rescue_cost   pi*(H)      pi*(L)")
print("----   ----------    --------    --------")

# Dla każdej pary (beta, rescue_cost) sprawdzamy, jaka polityka jest optymalna
for beta in beta_list:
    for rescue_cost in rescue_list:

        # budujemy świat (MDP) z tymi parametrami
        P, _, _ = build_recycling_robot_P(alpha=alpha, beta=beta,
                                          r_search=r_search,
                                          r_wait=r_wait,
                                          rescue_cost=rescue_cost)

        # znajdujemy politykę optymalną π* (brute force)
        pi_star, _ = best_policy_robot(P, gamma=gamma)

        # odczytujemy decyzje w H i L
        aH = action_name[int(np.argmax(pi_star[0]))]
        aL = action_name[int(np.argmax(pi_star[1]))]

        # wypisujemy wynik
        print(f"{beta:0.1f}     {rescue_cost:>10.1f}    {aH:<8}   {aL:<8}")

print("\nWskazówka: obserwuj, kiedy π*(L) zmienia się z SEARCH na RECHARGE.")


Ustawienia stałe: alpha= 0.8 r_search= 5.0 r_wait= 1.0 gamma= 0.9
beta   rescue_cost   pi*(H)      pi*(L)
----   ----------    --------    --------
0.1           -1.0    SEARCH     RECHARGE
0.1           -3.0    SEARCH     RECHARGE
0.1           -6.0    SEARCH     RECHARGE
0.1          -10.0    SEARCH     RECHARGE
0.3           -1.0    SEARCH     RECHARGE
0.3           -3.0    SEARCH     RECHARGE
0.3           -6.0    SEARCH     RECHARGE
0.3          -10.0    SEARCH     RECHARGE
0.5           -1.0    SEARCH     SEARCH  
0.5           -3.0    SEARCH     RECHARGE
0.5           -6.0    SEARCH     RECHARGE
0.5          -10.0    SEARCH     RECHARGE
0.7           -1.0    SEARCH     SEARCH  
0.7           -3.0    SEARCH     RECHARGE
0.7           -6.0    SEARCH     RECHARGE
0.7          -10.0    SEARCH     RECHARGE
0.9           -1.0    SEARCH     SEARCH  
0.9           -3.0    SEARCH     SEARCH  
0.9           -6.0    SEARCH     SEARCH  
0.9          -10.0    SEARCH     SEARCH  

Wskazówka: 

**Komentarz (krótko):**
- `pi*(H)` wychodzi tu stale jako `SEARCH`.
- `pi*(L)` przełącza się między `SEARCH` i `RECHARGE` zależnie od `beta` i `rescue_cost`.
- Im mniejsze `beta` (bardziej ryzykowne SEARCH w L) lub im bardziej ujemny `rescue_cost`, tym częściej wygrywa `RECHARGE`.


### Notatki dla prowadzącego (A1.4)

- To jest kluczowy moment, by powiedzieć:
  **parametry świata ≠ polityka**, ale zmieniają ranking `Q(s,a)`.

- Warto poprosić studentów o interpretację 2–3 konkretnych wierszy:
  np.:
  - `beta = 0.5, rescue = -1` vs `beta = 0.5, rescue = -3`
  - `beta = 0.9, rescue = -10` (dlaczego SEARCH nadal wygrywa?)

- To bardzo naturalnie prowadzi do:
  *„jak znaleźć π* bez brute-force?”* → Dynamic Programming.


## B. Gridworld 5x5 (Example 3.5 vs 3.8)

Tutaj świadomie pokazujemy **dwa różne problemy**:
1) **Ewaluacja**: polityka równoprawdopodobna (equiprobable) i liczenie $v_\pi$.
2) **Optymalizacja**: liczenie $v_*$ i polityki optymalnej $\pi_*$.

Różnica: w (1) mamy uśrednianie po akcjach (bez `max`), a w (2) pojawia się `max/argmax`.


### Opis środowiska (skrót)

- Siatka 5x5, akcje: `0=UP`, `1=RIGHT`, `2=DOWN`, `3=LEFT`.
- Jeśli ruch wychodzi poza planszę: zostajemy w miejscu i dostajemy nagrodę `-1`.
- Są dwa specjalne pola `A` i `B`:
  - z `A` niezależnie od akcji przechodzimy do `A'` z nagrodą `+10`,
  - z `B` niezależnie od akcji przechodzimy do `B'` z nagrodą `+5`.
- W tym przykładzie używamy $gamma=0.9$.


### Ćwiczenie B1 — Zbuduj model `P` dla Gridworld (A/B teleport)

Zaimplementuj `build_gridworld_AB_P()` zwracające `(P, nS, nA, nrow, ncol)`.

Wskazówki:
- `nrow=ncol=5`, `nS=25`, `nA=4`.
- Akcja zawsze deterministyczna: w `P[s][a]` jest pojedynczy wpis z `p=1.0`.


In [76]:
def build_gridworld_AB_P():
    """Gridworld 5x5 z A/B teleportami (jak w rozdz. 3 książki)."""
    nrow, ncol = 5, 5                      # rozmiar planszy
    nS, nA = nrow * ncol, 4                # 25 stanów, 4 akcje
    P = {s: {a: [] for a in range(nA)} for s in range(nS)}  # P[s][a] -> lista wyników

    # pozycje specjalne: A->A' (+10), B->B' (+5)
    A, Aprime, reward_A = (0, 1), (4, 1), 10.0
    B, Bprime, reward_B = (0, 3), (2, 3), 5.0

    # mapowanie stan <-> (wiersz, kolumna)
    def s2pos(s): return (s // ncol, s % ncol)
    def pos2s(r, c): return r * ncol + c

    # akcje: 0=UP, 1=RIGHT, 2=DOWN, 3=LEFT (wektory ruchu)
    moves = {0: (-1, 0), 1: (0, 1), 2: (1, 0), 3: (0, -1)}

    for s in range(nS):
        r, c = s2pos(s)                    # gdzie jesteśmy na planszy?

        # TELEPORTY MAJĄ PRIORYTET: z A/B zawsze teleport, niezależnie od akcji
        if (r, c) == A:
            s2 = pos2s(*Aprime)
            for a in range(nA):
                P[s][a] = [(1.0, s2, reward_A, False)]
            continue

        if (r, c) == B:
            s2 = pos2s(*Bprime)
            for a in range(nA):
                P[s][a] = [(1.0, s2, reward_B, False)]
            continue

        # STANDARDOWE RUCHY: deterministycznie przesuwamy się zgodnie z akcją
        for a in range(nA):
            dr, dc = moves[a]
            r2, c2 = r + dr, c + dc

            # OFF-GRID: zostajemy w miejscu i dostajemy -1
            if (r2 < 0) or (r2 >= nrow) or (c2 < 0) or (c2 >= ncol):
                s2 = s
                reward = -1.0
            else:
                s2 = pos2s(r2, c2)
                reward = 0.0

            P[s][a] = [(1.0, s2, reward, False)]  # deterministyczne przejście

    return P, nS, nA, nrow, ncol


# --- testy podstawowe ---
P, nS, nA, nrow, ncol = build_gridworld_AB_P()
assert nS == 25 and nA == 4 and nrow == 5 and ncol == 5
for s in range(nS):
    for a in range(nA):
        assert len(P[s][a]) == 1
        p, s2, r, term = P[s][a][0]
        assert abs(p - 1.0) < 1e-12
        assert term is False
print("B1 tests passed ✅")


B1 tests passed ✅


In [77]:
# --- testy semantyczne (logika świata, nie tylko format) ---

P, nS, nA, nrow, ncol = build_gridworld_AB_P()

# pomocniczo: mapowanie (r,c) <-> s
def pos2s(r, c): 
    return r * ncol + c

A = (0, 1); Aprime = (4, 1); reward_A = 10.0
B = (0, 3); Bprime = (2, 3); reward_B = 5.0

sA = pos2s(*A)
sAprime = pos2s(*Aprime)
sB = pos2s(*B)
sBprime = pos2s(*Bprime)

# 1) Teleport A: niezależnie od akcji zawsze idziemy do A' z nagrodą +10
for a in range(nA):
    assert P[sA][a] == [(1.0, sAprime, reward_A, False)]

# 2) Teleport B: niezależnie od akcji zawsze idziemy do B' z nagrodą +5
for a in range(nA):
    assert P[sB][a] == [(1.0, sBprime, reward_B, False)]

# 3) Off-grid: z narożnika (0,0) ruch UP lub LEFT -> zostajemy i dostajemy -1
s00 = pos2s(0, 0)
UP, RIGHT, DOWN, LEFT = 0, 1, 2, 3

assert P[s00][UP]   == [(1.0, s00, -1.0, False)]
assert P[s00][LEFT] == [(1.0, s00, -1.0, False)]

# 4) Normalny ruch: z (0,0) ruch RIGHT -> (0,1) z nagrodą 0 (uwaga: (0,1) to A,
# ale test dotyczy przejścia Z (0,0), więc jest normalne)
s01 = pos2s(0, 1)
assert P[s00][RIGHT] == [(1.0, s01, 0.0, False)]

print("B1 semantic tests passed ✅")


B1 semantic tests passed ✅


In [78]:
P[sA][a]

[(1.0, 21, 10.0, False)]

> **Notatki dla prowadzącego (B1):**
> - Studenci często mylą priorytet teleportów vs ruchów — tu teleporty nadpisują wszystko.
> - Ten przykład dobrze ćwiczy "świat jako funkcja" (przepisy) zamieniany na tablicę przejść.


### Po co to ćwiczenie? (dydaktycznie)

To ćwiczenie uczy, jak zamienić opis słowny środowiska na formalny model MDP `P[s][a]`.
To kluczowy krok przed równaniami Bellmana:

- najpierw kodujemy **świat** (przepisy ruchu, teleporty, kary),
- potem, w kolejnych ćwiczeniach, liczymy **wartości** dla zadanej polityki (średnia) albo dla optymalnej (max).

Najważniejsza pułapka: **teleporty A/B mają priorytet** i nadpisują zwykły ruch — niezależnie od wybranej akcji.



### B2 — Polityka równoprawdopodobna (equiprobable) i $v_\pi$

Tu liczymy $v_\pi$ dla polityki, która w każdym stanie wybiera każdy kierunek z prawdopodobieństwem 0.25.
To jest klasyczny przykład **policy evaluation**.


In [79]:
# Budujemy model świata Gridworld (A/B teleporty)
P, nS, nA, nrow, ncol = build_gridworld_AB_P()

gamma = 0.9  # dyskontowanie (jak w przykładach z książki)

# Polityka równoprawdopodobna:
# w każdym stanie każda z 4 akcji ma prawdopodobieństwo 1/4
pi = np.ones((nS, nA), dtype=float) / nA

# Budujemy model przejść i nagród "widoczny" dla tej polityki
P_pi, r_pi = build_P_r_for_policy(P, pi)

# Dokładnie liczymy funkcję wartości v_pi (układ równań Bellmana)
v_pi = evaluate_policy_linear_system(P_pi, r_pi, gamma)

# Wyświetlamy v_pi jako tabelę 5x5 (jak w książce)
print("v_pi jako tabela 5x5 (zaokrąglenie):")
pretty_matrix_as_grid(v_pi, nrow, ncol, decimals=1)


v_pi jako tabela 5x5 (zaokrąglenie):
[[ 3.3  8.8  4.4  5.3  1.5]
 [ 1.5  3.   2.3  1.9  0.5]
 [ 0.1  0.7  0.7  0.4 -0.4]
 [-1.  -0.4 -0.4 -0.6 -1.2]
 [-1.9 -1.3 -1.2 -1.4 -2. ]]


> **Notatki dla prowadzącego (B2):**
> - To jest dobry moment na porównanie do książkowej tabeli: wartości powinny się zgadzać (po zaokrągleniu).
> - Podkreśl: tu nie ma żadnej optymalizacji — to wartości dla z góry danej, losowej polityki.


## B3 — v* i polityka optymalna π* (podgląd)
Teraz (tylko jako podgląd) policzymy $v_*$ oraz $\pi_*$.

**Uwaga:** algorytm (value iteration) omówimy formalnie w rozdziale 4 (programowanie dynamiczne).
W tym rozdziale traktujemy go jako *narzędzie*, żeby zobaczyć różnicę między $v_\pi$ i $v_*$.
### Opis zadania
W tym ćwiczeniu **po raz pierwszy rozwiązujemy problem optymalny**:
nie oceniamy już danej polityki, lecz szukamy **najlepszych możliwych decyzji**
w każdym stanie Gridworldu.

Robimy to przy pomocy algorytmu *value iteration*, traktując go tutaj
wyłącznie jako **narzędzie poglądowe**.

Celem jest zobaczenie różnicy między:
- wartością dla danej polityki (`v_π`, ćwiczenie B2),
- wartością optymalną (`v*`) i polityką optymalną (`π*`).

---

### Notatka dla prowadzącego
- Tu **pojawia się `max`** — to jest równanie optymalności Bellmana.
- Warto jasno powiedzieć:
  *to jest już inny problem niż policy evaluation*.
- Jeśli padnie pytanie „czemu używamy DP już teraz”:
  → odpowiedź: **żeby zobaczyć efekt**, a formalną teorię robimy w rozdziale 4.


In [75]:
# B3 — v* i π* (podgląd różnicy względem v_pi)

def value_iteration(P, nS, nA, gamma=0.9, tol=1e-10, max_iter=100000):
    # Value iteration: rozwiązujemy równanie optymalności Bellmana
    V = np.zeros(nS, dtype=float)

    for _ in range(max_iter):
        delta = 0.0
        V_new = V.copy()

        for s in range(nS):
            best = None  # najlepsza wartość po akcjach

            for a in range(nA):
                outcomes = P[s][a]
                if not outcomes:
                    continue

                # liczymy Q(s,a)
                q = 0.0
                for (p, s2, r, terminated) in outcomes:
                    q += p * (r + gamma * (0.0 if terminated else V[int(s2)]))

                # wybieramy maksimum (tu jest „greedy”)
                if best is None or q > best:
                    best = q

            V_new[s] = 0.0 if best is None else best
            delta = max(delta, abs(V_new[s] - V[s]))

        V = V_new
        if delta < tol:
            break

    return V


def greedy_policy_from_V(P, nS, nA, V, gamma=0.9):
    # Odczytujemy π* jako argmax_a Q(s,a) względem V*
    pi_det = np.zeros(nS, dtype=int)

    for s in range(nS):
        best_a = 0
        best_q = None

        for a in range(nA):
            outcomes = P[s][a]
            if not outcomes:
                continue

            q = 0.0
            for (p, s2, r, terminated) in outcomes:
                q += p * (r + gamma * (0.0 if terminated else V[int(s2)]))

            if best_q is None or q > best_q:
                best_q = q
                best_a = a

        pi_det[s] = best_a

    return pi_det


# budujemy Gridworld (A/B teleporty)
P, nS, nA, nrow, ncol = build_gridworld_AB_P()
gamma = 0.9

# liczymy wartość optymalną v*
V_star = value_iteration(P, nS, nA, gamma=gamma)

# odczytujemy politykę optymalną π*
pi_star = greedy_policy_from_V(P, nS, nA, V_star, gamma=gamma)

# wizualizacja wyników
print("V* jako tabela 5x5 (zaokrąglenie):")
pretty_matrix_as_grid(V_star, nrow, ncol, decimals=1)

print("\nPolityka optymalna (strzałki):")
action_arrows(pi_star, nrow, ncol)


V* jako tabela 5x5 (zaokrąglenie):
[[24.5 27.2 24.5 23.6 21.2 19.1]
 [22.  24.5 23.6 26.2 23.6 21.2]
 [21.2 23.6 26.2 29.1 26.2 23.6]
 [19.1 21.2 23.6 26.2 23.6 21.2]
 [17.2 19.1 21.2 23.6 21.2 19.1]
 [15.5 17.2 19.1 21.2 19.1 17.2]]

Polityka optymalna (strzałki):
→ ↑ ← ↓ ↓ ↓
↑ ↑ → ↓ ↓ ↓
→ → → ↑ ← ←
↑ ↑ ↑ ↑ ↑ ↑
↑ ↑ ↑ ↑ ↑ ↑
↑ ↑ ↑ ↑ ↑ ↑


> **Notatki dla prowadzącego (B3):**
> - Warto jasno powiedzieć: tu wchodzi `max` (Bellman optimality). To jest inny problem niż B2.
> - Jeśli ktoś pyta "czemu używamy DP już teraz": odpowiedź: jako podgląd różnicy, a teorię robimy w rozdz. 4.
> - Tutaj nie oceniamy już konkretnej strategii, tylko rozwiązujemy problem decyzyjny: w każdym stanie wybieramy akcję, która maksymalizuje długoterminową wartość.

### B2 vs B3 — o co chodzi w różnicy?

- **B2 (policy evaluation)**: liczymy `v_π` dla *z góry zadanej* polityki `π`.  
  W równaniu Bellmana pojawia się **średnia po akcjach** (bo polityka losuje akcje).

- **B3 (optymalność)**: liczymy `v*` i `π*`, czyli najlepsze możliwe zachowanie.  
  W równaniu Bellmana pojawia się **max po akcjach** (bo wybieramy najlepszą decyzję).

Jedno zdanie intuicji:  
> W B2 pytamy „jak dobra jest dana strategia?”, a w B3 pytamy „jaka strategia jest najlepsza?”.



### Przejście do rozdziału 4 (Dynamic Programming)

W B3 użyliśmy value iteration jako narzędzia, żeby zobaczyć efekt optymalności;  
w rozdziale 4 pokażemy formalnie, **skąd bierze się ten algorytm, dlaczego działa i jak systematycznie znajduje `v*` oraz `π*`.**


### Ćwiczenia tablicowe (z książki): 3.14–3.16

Na zajęciach (bez kodu) przerobimy wybrane zadania 3.14–3.16. W notebooku zostawiamy tylko krótką listę kontrolną:

- (3.14) sprawdź/wyprowadź postać równań Bellmana w małym MDP,
- (3.15) interpretacja $v_\pi$ i wpływu $\gamma$,
- (3.16) różnica między $v_\pi$ a $v_*$ oraz rola `max/argmax`.

Rozwiązania: na tablicy oraz w materiałach prowadzącego.


## C. (Opcjonalnie) FrozenLake jako MDP

FrozenLake będzie naszym środowiskiem bazowym w kolejnych rozdziałach.
Tu tylko pokazujemy, że:
- FrozenLake udostępnia model przejść `env.unwrapped.P` w dokładnie tym samym formacie,
- więc możemy policzyć $v_\pi$ "modelowo" (gdy znamy `P`).


In [42]:
# Uwaga: ta sekcja jest opcjonalna. Jeśli nie masz gymnasium, pomiń.
try:
    import gymnasium as gym
except Exception as e:
    print("Brak gymnasium:", e)
    gym = None

if gym is not None:
    env = gym.make("FrozenLake-v1", is_slippery=False)
    P = env.unwrapped.P
    nS = env.observation_space.n
    nA = env.action_space.n
    gamma = 0.9

    # Przykładowa polityka: zawsze w prawo (RIGHT=1)
    pi = np.zeros((nS, nA))
    pi[:, 1] = 1.0

    P_pi, r_pi = build_P_r_for_policy(P, pi)
    v = evaluate_policy_linear_system(P_pi, r_pi, gamma)

    print("FrozenLake 4x4: v_pi (zawsze RIGHT), jako siatka:")
    pretty_matrix_as_grid(v, 4, 4, decimals=2)

    # Podgląd przejść dla stanu 0 i akcji RIGHT
    print("\nPrzykład P[0][RIGHT]:", P[0][1])


FrozenLake 4x4: v_pi (zawsze RIGHT), jako siatka:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

Przykład P[0][RIGHT]: [(1.0, 4, 0.0, False)]


In [43]:
P

{0: {0: [(1.0, 0, 0.0, False)],
  1: [(1.0, 4, 0.0, False)],
  2: [(1.0, 1, 0.0, False)],
  3: [(1.0, 0, 0.0, False)]},
 1: {0: [(1.0, 0, 0.0, False)],
  1: [(1.0, 5, 0.0, True)],
  2: [(1.0, 2, 0.0, False)],
  3: [(1.0, 1, 0.0, False)]},
 2: {0: [(1.0, 1, 0.0, False)],
  1: [(1.0, 6, 0.0, False)],
  2: [(1.0, 3, 0.0, False)],
  3: [(1.0, 2, 0.0, False)]},
 3: {0: [(1.0, 2, 0.0, False)],
  1: [(1.0, 7, 0.0, True)],
  2: [(1.0, 3, 0.0, False)],
  3: [(1.0, 3, 0.0, False)]},
 4: {0: [(1.0, 4, 0.0, False)],
  1: [(1.0, 8, 0.0, False)],
  2: [(1.0, 5, 0.0, True)],
  3: [(1.0, 0, 0.0, False)]},
 5: {0: [(1.0, 5, 0, True)],
  1: [(1.0, 5, 0, True)],
  2: [(1.0, 5, 0, True)],
  3: [(1.0, 5, 0, True)]},
 6: {0: [(1.0, 5, 0.0, True)],
  1: [(1.0, 10, 0.0, False)],
  2: [(1.0, 7, 0.0, True)],
  3: [(1.0, 2, 0.0, False)]},
 7: {0: [(1.0, 7, 0, True)],
  1: [(1.0, 7, 0, True)],
  2: [(1.0, 7, 0, True)],
  3: [(1.0, 7, 0, True)]},
 8: {0: [(1.0, 8, 0.0, False)],
  1: [(1.0, 12, 0.0, True)],
  2: [(

> **Notatki dla prowadzącego (C):**
> - Ta sekcja ma być krótka: tylko pokazuje, że Gym daje dostęp do `P`.
> - Na kolejnych zajęciach wrócimy do FrozenLake, gdy zaczniemy DP i RL.


## D. (Opcjonalnie) Pole balancing (CartPole) jako MDP

CartPole to też MDP, ale stan jest **ciągły** (wektor 4 liczb), więc tablicowe metody z tego rozdziału nie skalują.
To dobry most do późniejszych tematów (aproksymacja funkcji / deep RL).

Ten fragment nie jest ćwiczeniem — to tylko podgląd, jak wygląda MDP z ciągłym stanem.

In [41]:
# Opcjonalny podgląd: przestrzenie stanu i akcji w CartPole
try:
    import gymnasium as gym
except Exception as e:
    print("Brak gymnasium:", e)
    gym = None

if gym is not None:
    env = gym.make("CartPole-v1")
    obs, info = env.reset(seed=0)
    print("Observation (stan) =", obs)
    print("Observation space =", env.observation_space)
    print("Action space =", env.action_space)
    env.close()


Observation (stan) = [ 0.01369617 -0.02302133 -0.04590265 -0.04834723]
Observation space = Box([-4.8               -inf -0.41887903        -inf], [4.8               inf 0.41887903        inf], (4,), float32)
Action space = Discrete(2)


## Podsumowanie

- Umiesz zakodować model MDP jako `P[s][a]`.
- Umiesz policzyć $v_\pi$ dokładnie przez rozwiązanie układu liniowego.
- Widzisz różnicę: **ewaluacja** ($v_\pi$) vs **optymalność** ($v_*$, $\pi_*$).
- W kolejnych zajęciach (rozdz. 4) pokażemy, jak systematycznie liczyć $v_*$ i $\pi_*$ metodami DP.
